In [ ]:
import numpy as np
import struct 
import os
import plotly.graph_objects as go
import time

import lgchimera.general as general
from lgchimera.io import read_lidar_bin, read_gt
from lgchimera.kitti_util import process_kitti_gt, load_icp_results
from lgchimera.geom_util import euler_to_R

import symforce
try:
    symforce.set_epsilon_to_symbol()
except symforce.AlreadyUsedEpsilon:
    print("Already set symforce epsilon")
    pass 

import symforce.symbolic as sf

%load_ext autoreload
%autoreload 2

np.set_printoptions(suppress=True, precision=3)

### Load KITTI data

In [ ]:
kitti_seq = '0027'
start_idx = 0

gtpath = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

N_POSES = 100

### ICP data

In [ ]:
data_path = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'results', 'p2pl_icp')
lidar_Rs, lidar_ts, positions, lidar_covariances = load_icp_results(data_path, start_idx=start_idx)

### Problem Setup 

In [ ]:
# DEBUG: use toy positions
# gt_enu = np.zeros((N_POSES,3))
# gt_enu[:,0] = np.arange(N_POSES)

# gt_Rs = N_POSES * [np.eye(3)]

In [ ]:
# # Ground-truth odometry
# # odom[i] transforms i to i+1
# gt_odom = [None] * (N_POSES - 1)
# for i in range(N_POSES - 1):
#     R = gt_Rs[i+1] @ gt_Rs[i].T
#     t = gt_enu[i+1] - R @ gt_enu[i]
#     gt_odom[i] = (R, t)

In [ ]:
# Lidar odometry
lidar_odom = [None] * (N_POSES - 1)
for i in range(N_POSES - 1):
    lidar_odom[i] = (lidar_Rs[i], lidar_ts[i])

In [ ]:
# Initial lidar odometry poses
init_poses = [None] * N_POSES

In [ ]:
# Satellite positions
# Uniformly arranged in a circle of radius R at constant altitude
R = 1e4
SAT_ALT = 1e4
N_SATS = 10
satpos_enu = np.zeros((N_SATS, 3))
satpos_enu[:,0] = np.cos(np.linspace(0, 2*np.pi, N_SATS, endpoint=False)) * R
satpos_enu[:,1] = np.sin(np.linspace(0, 2*np.pi, N_SATS, endpoint=False)) * R
satpos_enu[:,2] = SAT_ALT

# sats_trace = go.Scatter(x=satpos_enu[:,0], y=satpos_enu[:,1])
# fig = go.Figure(data=[sats_trace])
# fig.update_layout(width=1000, height=1000, xaxis_title='East [m]', yaxis_title='North [m]')
# fig.show()

In [ ]:
# Ranges
PR_SIGMA = 10
m_ranges = np.zeros((N_POSES, N_SATS))
for i in range(N_POSES):
    for j in range(N_SATS):
        m_ranges[i,j] = np.linalg.norm(gt_enu[i] - satpos_enu[j]) + np.random.normal(0, PR_SIGMA)

In [ ]:
# Odometry
m_odometry = lidar_odom

In [ ]:
# Sigmas
odom_diag_sigma = np.array([0.05, 0.05, 0.05, 0.2, 0.2, 0.2])
range_sigma = PR_SIGMA

### Construct factor graph

In [ ]:
from lgchimera.symforce.factor_graph import build_values, build_factors

values = build_values(N_POSES, satpos_enu, m_ranges, m_odometry, 
                        range_sigma, odom_diag_sigma)

factors = build_factors(N_POSES, N_SATS)

### Optimization

In [ ]:
from symforce.opt.optimizer import Optimizer

# Select the keys to optimize - the rest will be held constant
optimized_keys = [f"poses[{i}]" for i in range(N_POSES)]

# Create the optimizer
optimizer = Optimizer(
    factors=factors,
    optimized_keys=optimized_keys,
    # Return problem stats for every iteration
    debug_stats=True,
    # Customize optimizer behavior
    params=Optimizer.Params(verbose=True, initial_lambda=1e4, lambda_down_factor=0.5),
)

# Solve and return the result
result = optimizer.optimize(values)

In [ ]:
print(f"Num iterations: {len(result.iteration_stats) - 1}")
print(f"Final error: {result.error():.6f}")

In [ ]:
graph_positions = np.zeros((N_POSES, 3))
for i, pose in enumerate(result.optimized_values["poses"]):
    graph_positions[i] = pose.position().flatten()
    #print(f"poses {i}: t = {pose.position()}, R = {pose.rotation().to_tangent()}")

In [ ]:
gt_enu

In [ ]:
fgo_traj = go.Scatter(x=graph_positions[:,0], y=graph_positions[:,1], hovertext=np.arange(N_POSES), name='FGO trajectory')
gt_traj = go.Scatter(x=gt_enu[:N_POSES,0], y=gt_enu[:N_POSES,1], hovertext=np.arange(N_POSES), name='Ground-truth')
start = go.Scatter(x=[0], y=[0], name='Start', mode='markers', marker=dict(size=10, color='blue'), showlegend=False)
fig = go.Figure(data=[gt_traj, fgo_traj, start])
fig.update_layout(width=1000, height=1000, xaxis_title='East [m]', yaxis_title='North [m]')
# Move legend into plot
fig.update_layout(legend=dict(x=0.75, y=0.98), font=dict(size=18))
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig.update_xaxes(autorange=True)
fig.show()